In [17]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_collabo_emote', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_collabo_emote...


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46475)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", 

Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:23


In [16]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [20]:
item_meta = mysql.read_table(spark, "metainfo", "meta_vc_sales_items")
collabo_emote = item_meta.where((col("event_type") == "collaboration") & (col("event_name").like("%emote%"))) \
    .select("device", "product_id", "product_name", "event_name", "start_time", "end_time").distinct()

In [62]:
collabo_emote.where((col("device") == "PC") & (col("end_time") <= current_timestamp())).orderBy("start_time").toPandas()

,device,product_id,product_name,event_name,start_time,end_time
0,PC,itemdesc.16100250,Victory Dance - LEENALCHI x JINJO CREW,2021_emote_mar2,2021-03-13 02:00:00,2021-06-12 02:00:00
1,PC,itemdesc.13000312,VICTORY DANCE - 2 PHUT HON (KAIZ REMIX),202107_phao_emote,2021-07-07 02:00:00,2021-10-06 01:59:59
2,PC,itemdesc.16100302,Victory Dance - NEO's Crazy Night,202112_kakaofriend_emote,2021-12-08 02:00:00,2022-01-06 02:00:00
3,PC,itemdesc.16100303,Victory Dance - TUBE's Moves,202112_kakaofriend_emote,2021-12-08 02:00:00,2022-01-06 02:00:00
4,PC,itemdesc.16100306,Victory Dance - The Magic Bomb,202201_themagicbomb_emote,2022-02-02 02:00:00,2022-04-27 02:00:00
5,PC,itemdesc.16100311,Victory Dance - Samsara by Jane Kim Choreography,202202_samsara_emote,2022-02-23 02:00:00,2022-05-18 02:00:00
6,PC,itemdesc.16100323,Victory Dance - HANA,202204_xiajunsu_emote,2022-04-13 02:00:00,2022-07-06 02:00:00
7,PC,itemdesc.16100341,Victory Dance - Roll 'n Rock,202207_balibandits_emote,2022-07-20 02:00:00,2022-10-12 02:00:00


In [11]:
collabo_emote.where((col("device") == "PC") & (col("end_time") <= current_timestamp())).orderBy("start_time").show(truncate=False)

+------+-----------------+------------------------------------------------+-------------------------+-------------------+-------------------+
|device|product_id       |product_name                                    |event_name               |start_time         |end_time           |
+------+-----------------+------------------------------------------------+-------------------------+-------------------+-------------------+
|PC    |itemdesc.16100250|Victory Dance - LEENALCHI x JINJO CREW          |2021_emote_mar2          |2021-03-13 02:00:00|2021-06-12 02:00:00|
|PC    |itemdesc.13000312|VICTORY DANCE - 2 PHUT HON (KAIZ REMIX)         |202107_phao_emote        |2021-07-07 02:00:00|2021-10-06 01:59:59|
|PC    |itemdesc.16100302|Victory Dance - NEO's Crazy Night               |202112_kakaofriend_emote |2021-12-08 02:00:00|2022-01-06 02:00:00|
|PC    |itemdesc.16100303|Victory Dance - TUBE's Moves                    |202112_kakaofriend_emote |2021-12-08 02:00:00|2022-01-06 02:00:00|
|PC   

In [12]:
collabo_emote.where((col("device") == "CONSOLE") & (col("end_time") <= current_timestamp())).orderBy("start_time").show(truncate=False)

+-------+-----------------+------------------------------------------------+-------------------------+-------------------+-------------------+
|device |product_id       |product_name                                    |event_name               |start_time         |end_time           |
+-------+-----------------+------------------------------------------------+-------------------------+-------------------+-------------------+
|CONSOLE|itemdesc.16100250|Victory Dance - LEENALCHI x JINJO CREW          |2021_emote_mar2          |2021-03-18 05:00:00|2021-06-17 05:00:00|
|CONSOLE|itemdesc.13000312|VICTORY DANCE - 2 PHUT HON (KAIZ REMIX)         |202107_phao_emote        |2021-07-15 04:00:00|2021-10-14 03:59:59|
|CONSOLE|itemdesc.16100303|Victory Dance - TUBE's Moves                    |202112_kakaofriend_emote |2021-12-09 04:00:00|2022-01-07 04:00:00|
|CONSOLE|itemdesc.16100302|Victory Dance - NEO's Crazy Night               |202112_kakaofriend_emote |2021-12-09 04:00:00|2022-01-07 04:00:00|

In [23]:
pc_leenalchi = pre_load_data_mart("pc", "2021-03-13", "2021-06-12", "gcoin_use").where(col("product_id") == "itemdesc.16100250")
pc_phao = load_data_mart("pc", "2021-07-07", "2021-10-06", "gcoin_use").where(col("product_id") == "itemdesc.13000312")
pc_kakao_neo = load_data_mart("pc", "2021-12-08", "2022-01-06", "gcoin_use").where(col("product_id") == "itemdesc.16100302")
pc_kakao_tube = load_data_mart("pc", "2021-12-08", "2022-01-06", "gcoin_use").where(col("product_id") == "itemdesc.16100303")
pc_magicbomb = load_data_mart("pc", "2022-02-02", "2022-04-27", "gcoin_use").where(col("product_id") == "itemdesc.16100306")
pc_samsara = load_data_mart("pc", "2022-02-23", "2022-05-18", "gcoin_use").where(col("product_id") == "itemdesc.16100311")
pc_xia = load_data_mart("pc", "2022-04-13", "2022-07-06", "gcoin_use").where(col("product_id") == "itemdesc.16100323")
pc_rollnrock = load_data_mart("pc", "2022-07-20", "2022-10-12", "gcoin_use").where(col("product_id") == "itemdesc.16100341")

In [65]:
pc_phao.select(sum("paid_use"), sum(col("free_use") + col("paid_use"))).show(truncate=False)

+-------------+--------------------------+
|sum(paid_use)|sum((free_use + paid_use))|
+-------------+--------------------------+
|185726960    |222378600                 |
+-------------+--------------------------+



In [25]:
pc_leenalchi = pc_leenalchi.withColumn("within_1month", when(col("date") < "2021-04-13", "1month").otherwise("over_month"))
pc_phao = pc_phao.withColumn("within_1month", when(col("date") < "2021-08-07", "1month").otherwise("over_month"))
pc_kakao_neo = pc_kakao_neo.withColumn("within_1month", when(col("date") < "2022-01-08", "1month").otherwise("over_month"))
pc_kakao_tube = pc_kakao_tube.withColumn("within_1month", when(col("date") < "2022-01-08", "1month").otherwise("over_month"))
pc_magicbomb = pc_magicbomb.withColumn("within_1month", when(col("date") < "2022-03-02", "1month").otherwise("over_month"))
pc_samsara = pc_samsara.withColumn("within_1month", when(col("date") < "2022-03-23", "1month").otherwise("over_month"))
pc_xia = pc_xia.withColumn("within_1month", when(col("date") < "2022-05-13", "1month").otherwise("over_month"))
pc_rollnrock = pc_rollnrock.withColumn("within_1month", when(col("date") < "2022-08-20", "1month").otherwise("over_month"))

In [37]:
pc_leenalchi_use = pc_leenalchi.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
pc_phao_use = pc_phao.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
pc_kakao_neo_use = pc_kakao_neo.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
pc_kakao_tube_use = pc_kakao_tube.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
pc_magicbomb_use = pc_magicbomb.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
pc_samsara_use = pc_samsara.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
pc_xia_use = pc_xia.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
pc_rollnrock_use = pc_rollnrock.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [38]:
pc_collabo_emote = pc_leenalchi_use.unionByName(pc_phao_use).unionByName(pc_kakao_neo_use).unionByName(pc_kakao_tube_use) \
    .unionByName(pc_magicbomb_use).unionByName(pc_samsara_use).unionByName(pc_xia_use).unionByName(pc_rollnrock_use) \
    .withColumn("device", lit("pc"))

In [29]:
console_leenalchi = pre_load_data_mart("console", "2021-03-18", "2021-06-17", "gcoin_use").where(col("product_id") == "itemdesc.16100250")
console_phao = load_data_mart("console", "2021-07-15", "2021-10-14", "gcoin_use").where(col("product_id") == "itemdesc.13000312")
console_kakao_neo = load_data_mart("console", "2021-12-09", "2022-01-07", "gcoin_use").where(col("product_id") == "itemdesc.16100302")
console_kakao_tube = load_data_mart("console", "2021-12-09", "2022-01-07", "gcoin_use").where(col("product_id") == "itemdesc.16100303")
console_magicbomb = load_data_mart("console", "2022-02-02", "2022-04-28", "gcoin_use").where(col("product_id") == "itemdesc.16100306")
console_samsara = load_data_mart("console", "2022-02-24", "2022-05-19", "gcoin_use").where(col("product_id") == "itemdesc.16100311")
console_xia = load_data_mart("console", "2022-04-21", "2022-07-14", "gcoin_use").where(col("product_id") == "itemdesc.16100323")
console_rollnrock = load_data_mart("console", "2022-07-21", "2022-10-13", "gcoin_use").where(col("product_id") == "itemdesc.16100341")

In [31]:
console_leenalchi = console_leenalchi.withColumn("within_1month", when(col("date") < "2021-04-18", "1month").otherwise("over_month"))
console_phao = console_phao.withColumn("within_1month", when(col("date") < "2021-08-15", "1month").otherwise("over_month"))
console_kakao_neo = console_kakao_neo.withColumn("within_1month", when(col("date") < "2022-01-09", "1month").otherwise("over_month"))
console_kakao_tube = console_kakao_tube.withColumn("within_1month", when(col("date") < "2022-01-09", "1month").otherwise("over_month"))
console_magicbomb = console_magicbomb.withColumn("within_1month", when(col("date") < "2022-03-02", "1month").otherwise("over_month"))
console_samsara = console_samsara.withColumn("within_1month", when(col("date") < "2022-03-24", "1month").otherwise("over_month"))
console_xia = console_xia.withColumn("within_1month", when(col("date") < "2022-05-21", "1month").otherwise("over_month"))
console_rollnrock = console_rollnrock.withColumn("within_1month", when(col("date") < "2022-08-21", "1month").otherwise("over_month"))

In [39]:
console_leenalchi_use = console_leenalchi.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
console_phao_use = console_phao.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
console_kakao_neo_use = console_kakao_neo.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
console_kakao_tube_use = console_kakao_tube.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
console_magicbomb_use = console_magicbomb.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
console_samsara_use = console_samsara.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
console_xia_use = console_xia.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))
console_rollnrock_use = console_rollnrock.groupBy("product_id", "within_1month").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [50]:
console_phao_use.show(truncate=False)

+-----------------+-------------+--------+---------+
|product_id       |within_1month|paid_use|total_use|
+-----------------+-------------+--------+---------+
|itemdesc.13000312|over_month   |3066380 |3607800  |
|itemdesc.13000312|1month       |5669000 |6621000  |
+-----------------+-------------+--------+---------+



In [40]:
console_collabo_emote = console_leenalchi_use.unionByName(console_phao_use).unionByName(console_kakao_neo_use).unionByName(console_kakao_tube_use) \
    .unionByName(console_magicbomb_use).unionByName(console_samsara_use).unionByName(console_xia_use).unionByName(console_rollnrock_use) \
    .withColumn("device", lit("console"))

In [58]:
all_device_collabo_emote = pc_collabo_emote.unionByName(console_collabo_emote)

In [ ]:
all_device_collabo_emote.groupBy("device").agg(count("*")).show(truncate=False)

In [52]:
all_device_collabo_emote = all_device_collabo_emote.join(collabo_emote.select("product_id", "product_name").distinct(), "product_id")

In [57]:
collabo_emote.select("product_id", "product_name").distinct().show(truncate=False)

+-----------------+------------------------------------------------+
|product_id       |product_name                                    |
+-----------------+------------------------------------------------+
|itemdesc.16100303|Victory Dance - TUBE's Moves                    |
|itemdesc.16100311|Victory Dance - Samsara by Jane Kim Choreography|
|itemdesc.16100323|Victory Dance - HANA                            |
|itemdesc.13000627|VICTORY DANCE - BBOOM BBOOM                     |
|itemdesc.16100346|Victory Dance - BBoom BBoom (Part 2)            |
|itemdesc.13000711|VICTORY DANCE - EMETSOUND                       |
|itemdesc.16100302|Victory Dance - NEO's Crazy Night               |
|itemdesc.16100250|Victory Dance - LEENALCHI x JINJO CREW          |
|itemdesc.16100341|Victory Dance - Roll 'n Rock                    |
|itemdesc.16100345|Victory Dance - BBoom BBoom (Part 1)            |
|itemdesc.13000312|VICTORY DANCE - 2 PHUT HON (KAIZ REMIX)         |
|itemdesc.16100306|Victory Dance -

In [61]:
all_device_collabo_emote.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- within_1month: string (nullable = false)
 |-- paid_use: long (nullable = true)
 |-- total_use: long (nullable = true)
 |-- device: string (nullable = false)



In [55]:
mysql.drop_table("labs", "all_device_collabo_emote")

In [56]:
mysql.insert_table(all_device_collabo_emote, "labs", "all_device_collabo_emote")

In [ ]:
all_device_collabo_emote_pd = all_device_collabo_emote.toPandas()

In [ ]:
all_device_collabo_emote_pd

In [43]:
all_device_collabo_emote_pd.to_csv("./all_device_collabo_emote.csv", index=False)